<a href="https://colab.research.google.com/github/sonu020101/Amazon-Product-recommendation-using-BERT/blob/main/Comments_prediction_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 1.x
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime

TensorFlow 1.x selected.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install bert-tensorflow==1.0.1

     |████████████████████████████████| 71kB 2.1MB/s 


In [ ]:
import re
import os
import pandas as pd

def load_data(PATH):
  DATA = {}
  DATA["sentence"]=[]
  DATA["sentiment"]=[]
  DATA["polarity"]=[]
  with tf.gfile.GFile(PATH,'r') as f:
    FILE = f.readlines()
    for x in FILE:
        DATA["sentiment"].append(re.match(r"__\w+__(\d+)",x).group(1))
        i = re.findall(r" \w+",x) 
        DATA["sentence"].append(''.join(i))
        #Positive polarity = 1
        #Negative polarity = 0
        if '__label__2' in x:
          DATA["polarity"].append("1")
        else:
          DATA["polarity"].append("0")
  df = pd.DataFrame.from_dict(DATA)
  return df

path = r'/content/drive/My Drive/REVIEWS/train.ft.txt'
train_df = load_data(path)
path = r'/content/drive/My Drive/REVIEWS/test.ft.txt'
test_df = load_data(path)

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
df = pd.read_csv(r'/content/drive/My Drive/Projett/train1.csv')
df1 = pd.read_csv(r'/content/drive/My Drive/Projett/test1.csv')

In [ ]:
df = df.loc[:,~df.columns.str.contains('Unnamed')]
df1 = df1.loc[:,~df1.columns.str.contains('Unnamed')]
#df1.head()

In [ ]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

In [ ]:
# Set the output directory for saving model file
# Optionally, set a GCP bucket location

OUTPUT_DIR = 'anoop'#@param {type:"string"}
#@markdown Whether or not to clear/delete the directory and create a new one
DO_DELETE = False #@param {type:"boolean"}
#@markdown Set USE_BUCKET and BUCKET if you want to (optionally) store model output on GCP bucket.
USE_BUCKET = False #@param {type:"boolean"}
BUCKET = 'BUCKET_NAME' #@param {type:"string"}

if USE_BUCKET:
  OUTPUT_DIR = 'gs://{}/{}'.format(BUCKET, OUTPUT_DIR)
  from google.colab import auth
  auth.authenticate_user()

if DO_DELETE:
  try:
    tf.gfile.DeleteRecursively(OUTPUT_DIR)
  except:
    # Doesn't matter if the directory didn't exist
    pass
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))


***** Model output directory: anoop *****


In [ ]:
#train, test = download_and_load_datasets()

df1

,sentence,sentiment,polarity
0,Great CD My lovely Pat has one of the GREAT v...,2,1
1,One of the best game music soundtracks for a ...,2,1
2,Batteries died within a year I bought this ch...,1,0
3,works fine but Maha Energy is better Check ou...,2,1
4,Great for the non Reviewed quite a bit of the...,2,1
...,...,...,...
399995,Unbelievable In a Bad Way We bought this Thom...,1,0
399996,Almost Great Until it Broke My son recieved t...,1,0
399997,Disappointed I bought this toy for my son who...,1,0
399998,Classic Jessica Mitford This is a compilation...,2,1


In [ ]:
train = df.sample(10000)
test = df1.sample(10000)
train.head()

,sentence,sentiment,polarity
2558645,Horrible One of the worst books I ever to rea...,1,0
926725,Love it My kids and I really enjoy this show ...,2,1
3145673,Great book great historic content Easy read h...,2,1
2494356,Scent protection I already had one of these b...,2,1
2840970,horiable What a heap of crap it must be the o...,1,0


In [ ]:
train.columns


Index(['sentence', 'sentiment', 'polarity'], dtype='object')

In [ ]:
DATA_COLUMN = 'sentence'
LABEL_COLUMN = 'polarity'
label_list = [0, 1]

In [ ]:
# Use the InputExample class from BERT's run_classifier code to create examples from the data
train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None,text_a = x[DATA_COLUMN],text_b = None,label = x[LABEL_COLUMN]), axis = 1)
test_InputExamples = test.apply(lambda x: bert.run_classifier.InputExample(guid=None, text_a = x[DATA_COLUMN],text_b = None,label = x[LABEL_COLUMN]), axis = 1)

In [ ]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"
def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],tokenization_info["do_lower_case"]])
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)
tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [ ]:
!pip install --upgrade tb-nightly
tokenizer.tokenize("This here's an example of using the BERT tokenizer")#EXAMPLE TO SHOW WHAT TOKENIZER DOES

     |████████████████████████████████| 10.6MB 2.9MB/s 


['this',
 'here',
 "'",
 's',
 'an',
 'example',
 'of',
 'using',
 'the',
 'bert',
 'token',
 '##izer']

In [ ]:
MAX_SEQ_LENGTH = 128
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 10000


INFO:tensorflow:Writing example 0 of 10000


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: None


...

INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: 1 (id = 1)


INFO:tensorflow:label: 1 (id = 1)


In [ ]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,num_labels):
  """Creates a classification model."""
  bert_module = hub.Module(BERT_MODEL_HUB,trainable=True)
  bert_inputs = dict(input_ids=input_ids,input_mask=input_mask,segment_ids=segment_ids)
  bert_outputs = bert_module(inputs=bert_inputs,signature="tokens",as_dict=True)
  output_layer = bert_outputs["pooled_output"]
  hidden_size = output_layer.shape[-1].value
  output_weights = tf.get_variable("output_weights", [num_labels, hidden_size],initializer=tf.truncated_normal_initializer(stddev=0.02))
  output_bias = tf.get_variable("output_bias", [num_labels], initializer=tf.zeros_initializer())
  with tf.variable_scope("loss"):
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)
    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)# Convert labels into one-hot encoding
    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)
    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)


In [ ]:
# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""
    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]
    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    # TRAIN and EVAL
    if not is_predicting:
      (loss, predicted_labels, log_probs) = create_model(is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)
      train_op = bert.optimization.create_optimizer(loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)
      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        f1_score = tf.contrib.metrics.f1_score(label_ids,predicted_labels)
        auc = tf.metrics.auc(label_ids,predicted_labels)
        recall = tf.metrics.recall(label_ids,predicted_labels)
        precision = tf.metrics.precision(label_ids,predicted_labels) 
        true_pos = tf.metrics.true_positives(label_ids,predicted_labels)
        true_neg = tf.metrics.true_negatives(label_ids,predicted_labels)   
        false_pos = tf.metrics.false_positives(label_ids,predicted_labels)  
        false_neg = tf.metrics.false_negatives(label_ids,predicted_labels)
        return {
            "eval_accuracy": accuracy,
            "f1_score": f1_score,
            "auc": auc,
            "precision": precision,
            "recall": recall,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
        }
      eval_metrics = metric_fn(label_ids, predicted_labels)
      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,loss=loss,train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,loss=loss,eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)
      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn


In [ ]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.9
# Model configs
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100

In [ ]:
# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

In [ ]:
# Specify outpit directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(model_dir=OUTPUT_DIR,save_summary_steps=SAVE_SUMMARY_STEPS,save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [ ]:
model_fn = model_fn_builder(num_labels=len(label_list),learning_rate=LEARNING_RATE,num_train_steps=num_train_steps,num_warmup_steps=num_warmup_steps)
estimator = tf.estimator.Estimator(model_fn=model_fn,config=run_config,params={"batch_size": BATCH_SIZE})

INFO:tensorflow:Using config: {'_model_dir': 'anoop', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f76bf977f60>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': 'anoop', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f76bf977f60>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [ ]:
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(features=train_features,seq_length=MAX_SEQ_LENGTH,is_training=True,drop_remainder=False)

In [ ]:
print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training!
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saving checkpoints for 937 into anoop/model.ckpt.


INFO:tensorflow:Saving checkpoints for 937 into anoop/model.ckpt.


INFO:tensorflow:Loss for final step: 0.0050571077.


INFO:tensorflow:Loss for final step: 0.0050571077.


Training took time  0:25:02.334402


In [ ]:
test_input_fn = run_classifier.input_fn_builder(features=test_features,seq_length=MAX_SEQ_LENGTH,is_training=False,drop_remainder=False)

In [ ]:
estimator.evaluate(input_fn=test_input_fn, steps=None)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
/tensorflow-1.15.2/python3.6/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving dict for global step 937: auc = 0.93690664, eval_accuracy = 0.9369, f1_score = 0.93628186, false_negatives = 309.0, false_positives = 322.0, global_step = 937, loss = 0.25112337, precision = 0.9350545, recall = 0.93751264, true_negatives = 4733.0, true_positives = 4636.0


INFO:tensorflow:Saving dict for global step 937: auc = 0.93690664, eval_accuracy = 0.9369, f1_score = 0.93628186, false_negatives = 309.0, false_positives = 322.0, global_step = 937, loss = 0.25112337, precision = 0.9350545, recall = 0.93751264, true_negatives = 4733.0, true_positives = 4636.0


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 937: anoop/model.ckpt-937


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 937: anoop/model.ckpt-937


{'auc': 0.93690664,
 'eval_accuracy': 0.9369,
 'f1_score': 0.93628186,
 'false_negatives': 309.0,
 'false_positives': 322.0,
 'global_step': 937,
 'loss': 0.25112337,
 'precision': 0.9350545,
 'recall': 0.93751264,
 'true_negatives': 4733.0,
 'true_positives': 4636.0}

In [ ]:
def getPrediction(in_sentences):
  labels = ["Negative", "Positive"]
  input_examples = [run_classifier.InputExample(guid="", text_a = x, text_b = None, label = 0) for x in in_sentences] # here, "" is just a dummy label
  input_features = run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
  predictions = estimator.predict(predict_input_fn)
  return [(sentence, prediction['probabilities'], labels[prediction['labels']]) for sentence, prediction in zip(in_sentences, predictions)]

In [ ]:
def polarity_calculation(pred1, product):
    print("\n\nCalling get pred...")
    predictions = getPrediction(pred1)
    print("\n\nPredictions done!")
    predictions
    pol = []
    
    for i,j,k in predictions:
      pol.append(k)
    
    count = 0
    
    for i in pol:
        if i == 'Positive':
            count += 1
    
    if count!=0:
      avg = count/len(pol)
      avg = avg*100
    else:
      avg = 0
    
    d[f'{product}'] = avg
    pol.clear()

In [ ]:
!pip install pydrive
from pydrive.drive import GoogleDrive
from pydrive.auth import GoogleAuth
def drive_upload(name):
    settings_path = 'settings.yaml'
    gauth = GoogleAuth(settings_file=settings_path)
    gauth.LoadCredentialsFile("/content/drive/My Drive/Projett/mycreds1.txt")
    
    if gauth.credentials is None:
        gauth.LocalWebserverAuth()
    elif gauth.access_token_expired:
        gauth.Refresh()
    else:
        gauth.Authorize()
    
    gauth.SaveCredentialsFile("/content/drive/My Drive/Projett/mycreds1.txt")
    
    drive = GoogleDrive(gauth)
    path = (f"/content/{name}_recommendation.csv")
    print('Started!!')
    file_list = drive.ListFile({'q': "'1zH0L2gDTC9tZTdrz49j3BmtgnAe0M9QW' in parents and trashed=False"}).GetList()
    f = drive.CreateFile({'title': f"{name}_recommendation.csv", 'parents': [{'id': '1zH0L2gDTC9tZTdrz49j3BmtgnAe0M9QW'}]})
    file_name = f['title']
    print('File name read....\n')

    try:
        for f1 in file_list:
            if f1['title'] == file_name:
                f1.Delete()
                print('Match found and file deleted....\n')
    except:
        print('Unable to get file.')

    x = os.path.join(path).replace("\\", r"/")
    f.SetContentFile(x)
    f.Upload()
    print(f'{file_name} uploaded successfully!!')
    f = None

In [ ]:
import os
import pandas as pd
import operator

products = ['Masks', 'Gloves', 'Sanitizer', 'Thermometer']

for product in products:
    l = []
    DAT = pd.read_csv(r'/content/drive/My Drive/Projett/{}.csv'.format(product))
    DAT = DAT.loc[:, ~DAT.columns.str.contains('Unnamed')]
    print(DAT.head())
    DAT = DAT.to_dict()
    DAT.keys()

    for i in DAT.keys():
        l.append(i)

    print(l)
    d = {}
    pred = []

    for i in range(len(l)):
        x = DAT[f'{l[i]}']
        print(x)
        
        for j in x:
            pred.append(str(x[j]))

        print('\n\nCalculating polarity...')
        polarity_calculation(pred,l[i])
        pred.clear()
    
    sorted_d = dict( sorted(d.items(), key=operator.itemgetter(1),reverse=True))
    sorted_d = pd.DataFrame(sorted_d.items())
    sorted_d.head()
    sorted_d = sorted_d.rename(columns={0:'Products',1:'Ratings'})
    print(sorted_d)
    if os.path.exists(f"/content/{product}_recommendation.csv"):
        os.remove(f"{product}_recommendation.csv")
    sorted_d.to_csv(f"{product}_recommendation.csv")
    drive_upload(product)

  Van Heusen Men's Cotton Face Mask (Pack of 4) (VHMSBRGPH53287_Assorted_Free Size)  ... Wildcraft W95 Plus Reusable Outdoor Respirator Supermask with Neckband (Medium 40-70 kg, Grindle : Turkish Blue) - Pack of 3
0  The total design shown in the pic doesn't matc...                                 ...  I am happy with the quality but not satisfied ...                                                                          
1  Good quality mask which comes with an embedded...                                 ...  Product is good but useful. Very short lent of...                                                                          
2        Design is ok but little tight for big face.                                 ...  Girst time its ok.but after u wash get hard br...                                                                          
3  Product is not exactly as depicted image but i...                                 ...  Mask material is ok.\nEar loops are very hurti...     

INFO:tensorflow:Writing example 0 of 8


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/__init__.py", line 36, in autodetect
    from google.appengine.api import memcache
ModuleNotFoundError: No module named 'google.appengine'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.locked_file'

During handl



Predictions done!
                                            Products  Ratings
0  HTC Instrument 103-CTH Digital Indoor Hygromet...     62.5
1                  Fluke 362, 200A AC/DC Clamp Meter     62.5
2  Omron MC 246 Digital Thermometer With Quick Me...     62.5
3  Smile Mom Digital Infrared Forehead Thermomete...     37.5
4  Digital Thermometer with LCD for Fridges Freez...     37.5
5            AGARO DT-555 Digital Themometer (White)     37.5
6  TCL Non-Touch 3-in-1 Digital Infrared Forehead...     25.0
7  Vandelay Infrared Thermometer -3 years Sensor ...     25.0
8  Dr Trust (USA) Forehead Digital Infrared Therm...     12.5
Started!!
File name read....

Match found and file deleted....

Thermometer_final.csv uploaded successfully!!


In [ ]:
import pandas as pd 
df = pd.read_csv(r'/content/drive/My Drive/Projett/Gloves.csv')
df = df.loc[:, ~df.columns.str.contains('Unnamed')]
df.rename(columns={'0':'Products','1':'Rating'}, inplace=True)

                                             Products  Rating
0   SOBBY Microfibre Wash and Dust Chenille Mitt C...   100.0
1   URBANMAC Dishwashing Gloves with Wash Scrubber...    75.0
2   Ansell Micro Touch N30 Nitrile Multipurpose Gl...    62.5
3   BO Unisex Reusable Washable Cotton Knitted Glo...    62.5
4   BO Unisex Reusable Washable Cotton Knitted Glo...    62.5
5   Rudham Disposable Gloves, 300 Pieces, Transparent    50.0
6   Max Home Magic Silicone Dish Washing Hand Glov...    50.0
7   Unieco Men Women Warm Woolen Winter Thermal Kn...    50.0
8   Kodenipr Club 100 Disposable Stretchable Blue ...    25.0
9   Bon Organik Reusable Cotton Gloves (Pack Of 10...    25.0
10  Handcuffs Biking/Cycling Gloves Water Resistan...    25.0
11  Careway Latex Medical Examination Disposable H...    12.5
12  Careway Latex Medical Examination Disposable P...    12.5
13  ASGARD® 3 Layer Face Protection Masks with NOS...    12.5
14  bvslf 300 Pcs Transparent Disposable Clear Pla...    12.5
